# Description
This Notebook provides an example for orbit determination based on laser ranging data.

We initialize the Java VM needed for Orekit, and test it by printing the Java version.

In [1]:
import orekit
vm = orekit.initVM()
print ('Java version:',vm.java_version)


Java version: 1.8.0_152-release


We then import the Orekit data and print the Earth equatorial radius.

In [2]:
from orekit.pyhelpers import setup_orekit_curdir
setup_orekit_curdir()
from org.orekit.utils import Constants
print (Constants.WGS84_EARTH_EQUATORIAL_RADIUS)

6378137.0


Configuring the account to use laser ranging data from the EDC TUM service. You need an account, it can be created for free at https://edc.dgfi.tum.de/en/register/
The API is described at https://edc.dgfi.tum.de/en/api/doc/python/

In [6]:
import getpass
username = 'jonglez' # Modify with your own username
password = getpass.getpass() # You will get prompted for your password
url = 'https://edc.dgfi.tum.de/api/v1/'

········


Listing the satellites available in the service

In [9]:
import requests
import json

args = {}
""" required options """
args['username'] = username
args['password'] = password
args['action'] = 'list-satellites'

""" optional options """
args['status'] = 'present'

""" send request as method POST """
response = requests.post(url, data=args)

if response.status_code == 200:
	""" convert json string in python list """
	data = json.loads(response.text)
	print(json.dumps(data, indent=4, sort_keys=True))
else:
	print(response.status_code)
	print(response.text)

[
    {
        "bin_size": null,
        "ext": "a11",
        "norad_number": null,
        "np_indicator": "2",
        "satellite_id": "0000100",
        "satellite_name": "apollo11",
        "sic_code": "0100",
        "status": "present"
    },
    {
        "bin_size": null,
        "ext": "l17",
        "norad_number": null,
        "np_indicator": "2",
        "satellite_id": "0000101",
        "satellite_name": "luna17",
        "sic_code": "0101",
        "status": "present"
    },
    {
        "bin_size": null,
        "ext": "a14",
        "norad_number": null,
        "np_indicator": "2",
        "satellite_id": "0000102",
        "satellite_name": "apollo14",
        "sic_code": "0102",
        "status": "present"
    },
    {
        "bin_size": null,
        "ext": "a15",
        "norad_number": null,
        "np_indicator": "2",
        "satellite_id": "0000103",
        "satellite_name": "apollo15",
        "sic_code": "0103",
        "status": "present"
    },
    

Getting information about Technosat, which bears the COSPAR ID 1704205. See the available satellites at https://edc.dgfi.tum.de/en/satellites/, or in the JSON list printed above.

In [10]:
cosparId = '1704205' 

args = {}
""" required options """
args['username'] = username
args['password'] = password
args['action'] = 'satellite-info'

""" optional options """
args['satellite_id'] = cosparId
#args['satellite_name'] = 'technosat'

""" send request as method POST """
response = requests.post(url, data=args)

if response.status_code == 200:
	""" convert json string in python list """
	data = json.loads(response.text)	
	for key in data.keys():
		print(key,':',data[key])
else:
	print(response.status_code)
	print(response.text)



status : present
np_indicator : 3
satellite_name : technosat
end_date : None
bin_size : 15
nice_name : Technosat
start_date : None
satellite_id : 1704205
ext : tec
launch_date : 2017-07-14
sic_code : 6203
norad_number : 42829
decay_date : None


Listing the ground stations.

In [16]:
args = {}
""" required options """
args['username'] = username
args['password'] = password
args['action'] = 'list-stations'

""" optional options """
args['active'] = 'yes'

""" send request as method POST """
response = requests.post(url, data=args)

if response.status_code == 200:
	""" convert json string in python list """
	data = json.loads(response.text)
	print(json.dumps(data, indent=4, sort_keys=True))
else:
	print(response.status_code)
	print(response.text)

[
    {
        "active": "yes",
        "code": "GLSL",
        "datacenter": "EDC",
        "latitude": "50.3633",
        "longitude": "30.4961",
        "site": "Golosiiv, Ukraine",
        "station_id": "1824"
    },
    {
        "active": "yes",
        "code": "MAIL",
        "datacenter": "EDC",
        "latitude": "38.6849",
        "longitude": "66.94309",
        "site": "Maidanak 1, Uzbekistan",
        "station_id": "1864"
    },
    {
        "active": "yes",
        "code": "KOML",
        "datacenter": "EDC",
        "latitude": "50.69461",
        "longitude": "136.74383",
        "site": "Komsomolsk-na-Amure, Russia",
        "station_id": "1868"
    },
    {
        "active": "yes",
        "code": "SIML",
        "datacenter": "EDC",
        "latitude": "44.4128",
        "longitude": "33.9931",
        "site": "Simeiz, Ukraine",
        "station_id": "1873"
    },
    {
        "active": "yes",
        "code": "MDVS",
        "datacenter": "EDC",
        "latitude

Getting detailed information about a ground station

In [12]:
args = {}
""" required options """
args['username'] = username
args['password'] = password
args['action'] = 'station-info'
args['station_id'] = '7080'

""" send request as method POST """
response = requests.post(url, data=args)

if response.status_code == 200:
	""" convert json string in python list """
	data = json.loads(response.text)	
	for key in data.keys():
		print(key,':',data[key])
else:
	print(response.status_code)
	print(response.text)

datacenter : CDDIS
code : MDOL
longitude : 255.9848
site : McDonald Observatory, Texas
station_id : 7080
latitude : 30.6802
secondary_name : Anthony Garcia
primary_name : Jerry Wiant
secondary_mail : anthg@utexas.edu
primary_mail : jerryrw@utexas.edu
active : yes


Searching for datasets corresponding to the user criteria.

In [15]:
args = {}
""" required options """
args['username'] = username
args['password'] = password
args['action'] = 'data-query'
args['data_type'] = 'NPT' # Normal pointing data

""" optional options """
args['start_data_date'] = '2018-12%'
args['end_data_date'] = '2018-12%'
args['satellite'] = cosparId
#args['provider'] = 'COD'
#args['station'] = '78403501'

""" send request as method POST """
response = requests.post(url, data=args)

if response.status_code == 200:
	""" convert json string in python list """
	data = json.loads(response.text)
	print(json.dumps(data, indent=4, sort_keys=True))
else:
	print(response.status_code)
	print(response.text)

[
    {
        "end_data_date": "2018-12-01 12:39:55",
        "errors": "",
        "id": "1790842",
        "incoming_date": "2018-12-02 23:24:34",
        "incoming_filename": "7825_technosat_20181201_12_00.NPT",
        "observations": "6",
        "satellite": "1704205",
        "start_data_date": "2018-12-01 12:35:00",
        "station": "78259001",
        "status": "valid",
        "version": "00",
        "wavelength": "532.10"
    },
    {
        "end_data_date": "2018-12-01 13:53:35",
        "errors": "",
        "id": "1790375",
        "incoming_date": "2018-12-01 14:02:16",
        "incoming_filename": "7237_technosat_crd_20181201_13_00.npt",
        "observations": "7",
        "satellite": "1704205",
        "start_data_date": "2018-12-01 13:51:00",
        "station": "72371901",
        "status": "valid",
        "version": "00",
        "wavelength": "532.000"
    },
    {
        "end_data_date": "2018-12-01 14:12:12",
        "errors": "",
        "id": "1790437"